In [3]:
import pandas as pd
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 
# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
code_df.head()

,name,code
0,BYC,001460
1,CJ CGV,079160
2,DB손해보험,005830
3,DGB금융지주,139130
4,DSR제강,069730


In [2]:
import urllib.parse
import pandas as pd

MARKET_CODE_DICT = {
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
}

DOWNLOAD_URL = 'kind.krx.co.kr/corpgeneral/corpList.do'

def download_stock_codes(market=None, delisted=False):
    params = {'method': 'download'}

    if market.lower() in MARKET_CODE_DICT:
        params['marketType'] = MARKET_CODE_DICT[market]

    if not delisted:
        params['searchType'] = 13

    params_string = urllib.parse.urlencode(params)
    request_url = urllib.parse.urlunsplit(['http', DOWNLOAD_URL, '', params_string, ''])

    df = pd.read_html(request_url, header=0)[0]
    df.종목코드 = df.종목코드.map('{:06d}'.format)

    return df
kosdaq_stocks = download_stock_codes('kosdaq')
kosdaq_stocks.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,CJ오쇼핑,035760,텔레비전 방송업,"의류, 생활주방, 가전제품, 보석, 기타",1999-11-23,12월,허민회,http://www.cjmall.com,서울특별시
1,CMG제약,058820,의약품 제조업,의약품,2001-08-31,12월,이주형,http://www.skynewpharm.com,서울특별시
2,IBKS제8호스팩,277410,금융 지원 서비스업,기업인수합병,2017-09-29,12월,박광열,NaN,서울특별시
3,KTH,036030,상품 중개업,"K쇼핑, 콘텐츠유통, ICT",1999-12-24,12월,김태환,http://www.kthcorp.com,서울특별시
4,SBI인베스트먼트,019550,신탁업 및 집합투자업,벤처캐피탈,1989-09-06,12월,공동대표이사 이준효 공동대표이사 소우에이이치로,http://www.sbik.co.kr,서울특별시


In [5]:
from pandas_datareader import data

results = {}
for code in kosdaq_stocks.종목코드.head():
    results[code] = data.DataReader(code + '.KQ', 'iex', '2017-01-01', None)

df = pd.concat(results, axis=1)
df.loc[:, pd.IndexSlice[:, 'Adj Close']].tail()

2y


KeyError: '035760.KQ'